<img src="https://egc2023.sciencesconf.org/data/pages/logo_2.jpg" alt="EGC 2023" width="200px"/>


# Démonstration - Perdido Geoparser - EGC 2023 

Cette démonstration présente la librairie Python [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding de textes en français. 


**Auteurs** : [Ludovic Moncla](https://ludovicmoncla.github.io) (Univ Lyon, INSA Lyon, CNRS, UCBL, LIRIS, UMR 5205, F-69621)
[Mauro Gaio](https://lma-umr5142.univ-pau.fr/fr/organisation/membres/cv_-mgaio-fr.html) (Université de Pau et des Pays de l'Adour, CNRS, LMAP, UMR 5142)

## 1. En bref


Dans cette démonstration, nous allons voir comment :

- Utiliser la librarie [Perdido](https://github.com/ludovicmoncla/perdido) pour le geoparsing et le geocoding :
  - afficher les entités nommées annotées ;
  - cartographier les lieux geocodés ;
  - illustrer la problématique de désambiguïsation des toponymes.
  - enregistrer les résultats dans différents formats (csv, dataframe, ...)

## 2. Configurer l'environnement


* Si vous avez déjà configuré votre environnement, soit avec conda, soit avec pip (voir le fichier [README.md](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing/-/blob/main/README.md)), vous pouvez ignorer la section suivante et passer directement à la 3.
* Si vous exécutez ce notebook depuis Google Colab, vous devez exécuter la cellule suivante :

In [ ]:
! pip install perdido==0.1.33
! pip install stanza==1.4.2

In [21]:
import warnings
warnings.filterwarnings('ignore')

## 3. Perdido Geoparser

`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le **geotagging** et le **geocoding**. 

Le geotagging est similaire à la tâche de **reconnaissance des entités nommées** avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).

Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques non ambigues.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.

Dans cette partie nous allons voir comment utiliser `Perdido` pour le geoparsing.

### 3.1 Importer la librairie

In [1]:
from perdido.geoparser import Geoparser

### 3.2 Executer le geoparser

In [2]:
geoparser = Geoparser()
doc = geoparser('Je visite la ville de Lyon, Annecy et Chamonix.')

### 3.3 Visualiser les résultats

* Visualiser les attributs des tokens :

In [3]:
for token in doc:
    print(f'{token.text}\tlemma: {token.lemma}\tpos: {token.pos}')

Je	lemma: je	pos: PRO
visite	lemma: visiter	pos: V
la	lemma: le	pos: DET
ville	lemma: ville	pos: N
de	lemma: de	pos: PREP
Lyon	lemma: lyon	pos: NPr
,	lemma: 	pos: PUN
Annecy	lemma: annecy	pos: NPr
et	lemma: et	pos: CONJC
Chamonix	lemma: chamonix	pos: NPr
.	lemma: 	pos: SEN


* Format IOB :

In [4]:
for token in doc:
    print(token.iob_format())

Je je PRO O
visite visiter V O
la le DET B-LOC-NNE
ville ville N I-LOC-NNE
de de PREP I-LOC-NNE
Lyon lyon NPr I-LOC-NNE B-LOC
,  PUN O
Annecy annecy NPr B-LOC
et et CONJC O
Chamonix chamonix NPr B-LOC
.  SEN O


* Format IOB-TSV :

In [5]:
for token in doc:
    print(token.tsv_format())

1	Je	je	PRO	O
2	visite	visiter	V	O
3	la	le	DET	B-LOC-NNE
4	ville	ville	N	I-LOC-NNE
5	de	de	PREP	I-LOC-NNE
6	Lyon	lyon	NPr	I-LOC-NNE	B-LOC
7	,		PUN	O
8	Annecy	annecy	NPr	B-LOC
9	et	et	CONJC	O
10	Chamonix	chamonix	NPr	B-LOC
11	.		SEN	O


* Afficher la sortie XML-TEI :

In [6]:
print(doc.tei)

<TEI><teiheader/><text><body><s><w pos="PRO" lemma="je" subtype="PpvIL" id="w0" idx="0">Je</w><phr type="motion"><motionmedian><w pos="V" lemma="visiter" id="w1" idx="3">visite</w></motionmedian><rs type="ene" id="en.0"><rs type="place" subtype="ene" id="en.1" start="10" end="12" startT="2" endT="6"><term type="place" start="10" end="12" startT="2" endT="4"><w pos="DET" lemma="le" subtype="ART" id="w2" idx="10">la</w><w pos="N" lemma="ville" id="w3" idx="13">ville</w></term><w pos="PREP" lemma="de" id="w4" idx="19">de</w><rs type="place" subtype="no" id="en.2" start="22" end="26" startT="5" endT="6"><name type="place" id="en.3" start="22" end="26" startT="5" endT="6"><w pos="NPr" lemma="lyon" id="w5" idx="22">Lyon</w><location><geo source="nominatim" rend="Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, France">4.832011 45.757814</geo></location></name></rs></rs></rs><w pos="PUN" lemma="" id="w6" idx="26">,</w><rs type="place" subtype="no" id="en.4" start="

* Afficher la sortie GeoJSON :

In [7]:
print(doc.geojson)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [4.832011, 45.757814]}, 'properties': {'id': 'en.3', 'name': 'Lyon', 'sourceName': 'Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, France', 'type': 'administrative', 'country': 'France', 'source': 'nominatim'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [6.128885, 45.899235]}, 'properties': {'id': 'en.5', 'name': 'Annecy', 'sourceName': 'Annecy, Haute-Savoie, Auvergne-Rhône-Alpes, France métropolitaine, France', 'type': 'administrative', 'country': 'France', 'source': 'nominatim'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [6.872751, 45.92467]}, 'properties': {'id': 'en.7', 'name': 'Chamonix', 'sourceName': 'Chamonix-Mont-Blanc, Bonneville, Haute-Savoie, Auvergne-Rhône-Alpes, France métropolitaine, 74400, France', 'type': 'administrative', 'country': 'France', 'source': 'nominatim'}}]}


* Afficher la liste des entités nommées :

In [8]:
for entity in doc.named_entities:
    print(f'entity: {entity.text}\ttag: {entity.tag}')
    if entity.tag == 'place':
        for t in entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')

entity: Lyon	tag: place
 latitude: 4.832011	longitude: 45.757814	source nominatim
entity: Annecy	tag: place
 latitude: 6.128885	longitude: 45.899235	source nominatim
entity: Chamonix	tag: place
 latitude: 6.872751	longitude: 45.92467	source nominatim


* Afficher la liste des entités nommées étendues :

In [9]:
for nested_entity in doc.nested_named_entities:
    print(f'entity: {nested_entity.text}\ttag: {nested_entity.tag}')
    if nested_entity.tag == 'place':
        for t in nested_entity.toponym_candidates:
            print(f' latitude: {t.lat}\tlongitude: {t.lng}\tsource {t.source}')


entity: la ville de Lyon	tag: place
 latitude: 4.832011	longitude: 45.757814	source nominatim


* Affichage graphique des résultats avec la librairie [spacy](https://spacy.io/usage/visualizers) :

In [10]:
from spacy import displacy

In [11]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)

In [12]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

### 3.4 Exporter les résultats

* Enregistrer les résultats au format XML-TEI :

In [13]:
doc.to_xml('filename.xml')

* Enregistrer les résultats au format GeoJSON :

In [14]:
doc.to_geojson('filename.geojson')

* Enregistrer les résultats au format IOB-TSV :

In [15]:
doc.to_iob('filename.tsv')

* Enregistrer les résultats au format CSV :

In [16]:
doc.to_csv('filename.csv')

### 3.5 Paramétrage

La librairie est disponible en 2 versions : une version `Standard` et une version `Encyclopedie` spécialement adaptée pour les articles encyclopédiques. L'étape de géocoding est fortement paramétrable, en particulier afin de filtrer les résultats provenants des ressources géographiques (dans le but de limiter les ambiguïtés).

#### 3.5.1 Paramétrage du geotagging

* Paramétrer la version des règles d'annotation utilisée pour la reconnaissance des entités nommées :

    * `Standard` (par défaut): règles développées initialement pour le traitement de descriptions de randonnées
    * `Encyclopedie` : règles adaptées pour le traitement d'article encyclopédique

In [17]:
text = "ARQUES, (Géog.) petite ville de France, en Normandie, "
text += "au pays de Caux, sur la petite riviere d'Arques. Long. 18. 50. lat. 49. 54."

geoparser = Geoparser(version="Encyclopedie")
doc = geoparser(text)

displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

#### 3.5.2 Paramétrage du geocoding

* Pour les exemples suivants nous allons utiliser un extrait d'une description de randonnée :

In [30]:
text  =  "Départ du refuge d'Entre le Lac près du lac de la Plagne."
text += " Du refuge Entre le Lac, un sentier remonte les pentes herbeuses et permet de rejoindre le GR5 un peu avant le chalet de la Grassaz (chalet du berger 2335m)."
text += " Toujours en direction du sud , on remonte le vallon en longeant le ruisseau." 
text += " On parvient ainsi à l'extrémité ouest du lac de Grattaleu ; un peu plus haut, on atteint le refuge du col du Palet (2550m)."
text += " On admire la beauté de la vallée et le sommet de Bellecote recouvert de glaciers"

* Paramétrer la ou les ressources utilisées (gazetier) : 

    * `nominatim` (par défaut): [OpenStreetMap](https://www.openstreetmap.org)
    * `ign` : [GeoPortail](https://www.geoportail.gouv.fr)
    * `geonames` : [Geonames](http://www.geonames.org)
    * `whg`: [World Historical Gazetteer](https://whgazetteer.org)
    * `pleiades`: [Pleiades](https://pleiades.stoa.org)

In [28]:
geoparser = Geoparser(sources=['ign'])
doc = geoparser(text)
doc.get_folium_map()

* Paramétrer le nombre de résultats retournés pour chaque toponyme (par ressource), 1 par défaut :

In [29]:
geoparser = Geoparser(max_rows=10)
doc = geoparser(text)
doc.get_folium_map()

* Filtrer les résultats par pays (code pays) :

In [31]:
geoparser = Geoparser(max_rows=10, country_code = 'fr')
doc = geoparser(text)
doc.get_folium_map()

* Filtrer les résultats selon une zone géographique (bounding box: `east`,`south`,`west`,`north`) :

In [32]:
geoparser = Geoparser(max_rows=10, bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403])
doc = geoparser(text)
doc.get_folium_map()

### 3.6 Désambiguïsation

La librairie Perdido est toujours en cours de développement et d'amélioration dans le cadre de différents projet de recherche ([ANR CHOCUAS](), [GEODE]()), à l'heure actuelle une seule méthode de désambiguïsation automatique est disponible. Il s'agit d'une méthode de filtrage par clustering.


#### 3.6.1 Clustering par densité spatiale

Le principe est de regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*) et de selectionner le cluster qui contient le plus d'entités distinctes.

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 

Cette stratégie est adaptée pour des textes où les différents lieux cités sont supposés être localisés à proximité les uns des autres (ex: descriptions de randonnées).

In [34]:
geoparser = Geoparser()
doc = geoparser(text)

doc.cluster_disambiguation()

doc.get_folium_map()

## 4. Perdido Geocoder

En plus de la classe Geoparser, la librairie Perdido propose aussi la classe Geocoder. Cette classe permet de geocoder un ou plusieurs toponymes. Les paramètres sont les mêmes que ceux utilisés avec le geoparser pour configuer l'étape de geocoding :
* sources
* max_row
* country_code
* bbox

L'objet retourné est de type Perdido comme pour le Geoparser ce qui permet d'avoir accès aux mêmes attributs et méthodes que précédemment.

### 4.1 Importer la librairie

In [35]:
from perdido.geocoder import Geocoder

### 4.2 Executer le geocoder

* Instancier le geocoder :

In [36]:
geocoder = Geocoder()

* Geocoder un nom de lieu :

In [37]:
doc = geocoder('Lyon')

* Geocoder une liste de noms de lieux :

In [38]:
doc = geocoder(['Lyon', 'Annecy', 'Chamonix'])

### 4.3 Visualiser les résultats

* Afficher le résultat GeoJSON :

In [39]:
print(doc.geojson)

{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [4.832011, 45.757814]}, 'properties': {'id': 0, 'name': 'Lyon', 'sourceName': 'Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, France', 'type': 'administrative', 'country': 'France', 'source': 'nominatim'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [6.128885, 45.899235]}, 'properties': {'id': 0, 'name': 'Annecy', 'sourceName': 'Annecy, Haute-Savoie, Auvergne-Rhône-Alpes, France métropolitaine, France', 'type': 'administrative', 'country': 'France', 'source': 'nominatim'}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [6.872751, 45.92467]}, 'properties': {'id': 0, 'name': 'Chamonix', 'sourceName': 'Chamonix-Mont-Blanc, Bonneville, Haute-Savoie, Auvergne-Rhône-Alpes, France métropolitaine, 74400, France', 'type': 'administrative', 'country': 'France', 'source': 'nominatim'}}]}


* Afficher la liste des toponymes canddidats :

In [40]:
for t in doc.toponyms: 
    print(f'lat: {t.lat}\tlng: {t.lng}\tsource {t.source}\tsourceName {t.source_name}')

lat: 4.832011	lng: 45.757814	source nominatim	sourceName Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône-Alpes, France métropolitaine, France
lat: 6.128885	lng: 45.899235	source nominatim	sourceName Annecy, Haute-Savoie, Auvergne-Rhône-Alpes, France métropolitaine, France
lat: 6.872751	lng: 45.92467	source nominatim	sourceName Chamonix-Mont-Blanc, Bonneville, Haute-Savoie, Auvergne-Rhône-Alpes, France métropolitaine, 74400, France


* Récupérer les toponymes candidats sous la forme d'un geodataframe :

In [42]:
doc.to_geodataframe()

,geometry,id,name,sourceName,type,country,source
0,POINT (4.83201 45.75781),0,Lyon,"Lyon, Métropole de Lyon, Rhône, Auvergne-Rhône...",administrative,France,nominatim
1,POINT (6.12889 45.89923),0,Annecy,"Annecy, Haute-Savoie, Auvergne-Rhône-Alpes, Fr...",administrative,France,nominatim
2,POINT (6.87275 45.92467),0,Chamonix,"Chamonix-Mont-Blanc, Bonneville, Haute-Savoie,...",administrative,France,nominatim


* Afficher la carte des résultats

In [43]:
doc.get_folium_map()

## 5. Les jeux de données


La libraire [Perdido](https://github.com/ludovicmoncla/perdido) embarque deux jeux de données : 
 1. des articles encyclopédiques (volume 7 de l'Encyclopédie de Diderot et d'Alembert (1751-1772)), fournit par l'[ARTFL](https://encyclopedie.uchicago.edu) dans le cadre du projet [GEODE](https://geode-project.github.io) ;
 2. des descriptions de randonnées (chaque description est associée à sa trace GPS. Elles proviennent du site [www.visorando.fr](https://www.visorando.com) et ont été collectées dans le cadre du projet [ANR CHOUCAS](http://choucas.ign.fr).

### 5.1 Articles encyclopédiques

Le jeu de données des articles encyclopédiques est disponible dans la librairie en deux versions, une version "brute" (articles fournis par l'ARTFL) au format dataframe et une version déjà annotée par Perdido (format PerdidoCollection). 

%%Nous allons charger la version brute et voir comment manipuler un dataframe.

#### 5.1.1 Corpus brut

In [44]:
from perdido.datasets import load_edda_artfl 

dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']
data_artfl.head()

,filename,volume,number,head,normClass,author,text
0,volume07-1.tei,7,1,Title Page,unclassified,unsigned,"ENCYCLOPÉDIE, ou DICTIONNAIRE RAISONNÉ DES SCI..."
1,volume07-10.tei,7,10,FOESNE ou FOUANE,Marine | Pêche,Bellin,"FOESNE ou FOUANE, sub. s. (Marine & Pêche.) c'..."
2,volume07-100.tei,7,100,Fond de la hune,unclassified,Bellin,Fond de la hune ; ce sont les planches qu on p...
3,volume07-1000.tei,7,1000,Fronteau,Bourrelier | Sellier,Diderot,"* Fronteau, terme de Sellier-Bourrelier ; c'es..."
4,volume07-1001.tei,7,1001,FRONTIERE,Géographie,Diderot,"* FRONTIERE, s. f. (Géog.) se dit des limites,..."


In [45]:
data_artfl.loc[data_artfl['head'] == 'FRONTIGNAN'].text.item()

"FRONTIGNAN, (Géog.) petite ville de France. au Bas-Languedoc, connue par ses excellens vins muscats, & ses raisins de caisse qu'on appelle passerilles. Quelques savans croyent, sans en donner de preuves, que cette ville est le forum Domitii des Romains. Elle est située sur l'étang de Maguelone, à six lieues N. E. d'Agde, & cinq S. O. de Montpellier. Long. 15d. 24'. lat. 43d. 28'. (D. J.)"

#### 5.1.2 Corpus traité par Perdido

In [50]:
from perdido.datasets import load_edda_perdido

dataset_perdido = load_edda_perdido()
data_perdido = dataset_perdido['data']
df = data_perdido.to_dataframe()
df.head()

,filename,volume,number,head,normClass,author,text,#_places,#_person,#_event,#_date,#_misc,#_locations
0,volume07-1047.tei,07,1047,Fuego ou Fogo (Isle de-),Géographie,Jaucourt,"Fuego ou Fogo (Isle de-), Géog. cette seconde ...",9,2,2,2,2,70
1,volume07-1084.tei,07,1084,Fumeterre,Pharmacie. Matière médicale,Venel,"Fumeterre, (Pharmacie. Mat. med.) cette plante...",2,0,0,0,0,20
2,volume07-1090.tei,07,1090,FUMISTE,Art méchanique,unsigned,"FUMISTE, s. m. (Arts méc.) On appelle ainsi ce...",0,0,0,0,0,0
3,volume07-1127.tei,07,1127,FUNTA,Commerce,unsigned,"FUNTA, s. m. (Commerce.) poids dont on se sert...",2,1,0,0,0,5
4,volume07-1133.tei,07,1133,Fureur,Médecine,d'Aumont,"Fureur, (Medecine.) c'est un symptome qui est ...",1,0,0,0,0,6


In [55]:
doc = data_perdido[0]
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [56]:
doc.get_folium_map()

AttributeError: 'Perdido' object has no attribute 'geometry_layer'

### 5.2 Descriptions de randonnées (traitées par Perdido)

In [22]:
from perdido.datasets import load_choucas_perdido

dataset_choucas = load_choucas_perdido()
data_choucas = dataset_choucas['data']

df = data_choucas.to_dataframe()
df.head()

,name,text,geometry,#_places,#_person,#_event,#_date,#_misc,#_locations
0,Chalets de la Fullie,\n\nBoucle des chalets de la Fullie au départ ...,"(LINESTRING (6.11174 45.616041, 6.11174 45.616...",17,0,0,0,0,17
1,Traversée cabane de Pravouta à la Plagne,\n\nPartir de la cabane de Pravouta juste de l...,"(LINESTRING (5.832543 45.315222, 5.832444 45.3...",23,2,0,0,0,23
2,Refuge Entre Le Lac - Refuge de la Leisse,\n\nDépart du refuge d'Entre le Lac près du la...,"(LINESTRING (6.839184 45.480323, 6.83987 45.47...",22,0,0,0,0,22
3,Le lac du Retour,"\n\nDu parking de Pierre Giret, suivre la rout...","(LINESTRING (6.917631 45.619278, 6.917527 45.6...",6,1,0,0,0,6
4,Traversée Alpette - Dent de Crolles,\n,"(LINESTRING (5.907402 45.440585, 5.907439 45.4...",0,0,0,0,0,0


In [23]:
doc = data_choucas[2]

In [24]:
doc.text

"\n\nDépart du refuge d'Entre le Lac près du lac de la Plagne.\nDu refuge Entre le Lac, un sentier remonte les pentes herbeuses et permet de rejoindre le GR5 un peu avant le chalet de la Grassaz (chalet du berger 2335m). Toujours en direction du sud, on remonte le vallon en longeant le ruisseau. On parvient ainsi à l'extrémité ouest du lac de Grattaleu; un peu plus haut, on atteint le refuge du col du Palet (2550m). On admire la beauté de la vallée et le sommet de Bellecote recouvert de glaciers. Le GR descend vers l'Est; le sentier serpente entre des entonnoirs créés dans le gypse par dissolution. Le GR passe sous un 1er télésiège, celui de Grattaleu, et près de l'arrivée d'un second, le Tichot. Au chalet de Lognan (croix) prendre à droite un sentier qui descend à Val Claret (2107m) (station de ski). Poursuivre jusqu'au chalet de la Leisse. Le GR55 s'élève vers le vallon du paquis. On passe en contrebas du chalet du Prariond; un peu plus loin on arrive à la bifurcation du col de Fress

In [25]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [26]:
doc.get_folium_map()

## 6. Pour aller plus loin

1. Tutoriel (en français) Geoparsing : [https://github.com/ludovicmoncla/tutoriel-geoparsing](https://github.com/ludovicmoncla/tutoriel-geoparsing)
2. Tutoriel (en français) présenté lors de l'atelier [Librairies Python et Services Web pour la reconnaissance d’entités nommées et la résolution de toponymes](https://anf-tdm-2022.sciencesconf.org/resource/page/id/11) de la formation CNRS [ANF TDM 2022](https://anf-tdm-2022.sciencesconf.org) : [https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing](https://gitlab.liris.cnrs.fr/lmoncla/tutoriel-anf-tdm-2022-python-geoparsing)
3. Tutoriel (en anglais) utilisé pour le cours [SunoikisisDC](https://sunoikisisdc.github.io) Summer 2022 Course on [Natural Language Processing (NLP) for historical texts](https://github.com/SunoikisisDC/SunoikisisDC-2021-2022/wiki/SunoikisisDC-Summer-2022-Session-9) (Session 9)

* Charger le jeu de données :

In [1]:
dataset_artfl = load_edda_artfl()
data_artfl = dataset_artfl['data']

NameError: name 'load_edda_artfl' is not defined

* Afficher la liste des premiers articles :

In [ ]:
data_artfl.head()

* Récupération du contenu de l'article FRONTIGNAN :

In [ ]:
frontignan = data_artfl.loc[data_artfl['head'] == 'FRONTIGNAN'].text.item()
frontignan

## 4. Reconnaissance d'Entités Nommées (NER)

La reconnaissance d'entités nommées, *Named Entity Recognition* (NER) en anglais, est une tâche très importante et incontournable en traitement automatique des langues (TAL) et en compréhension du langage naturel (NLU en anglais). 
Cette tâche consiste à rechercher des objets textuels (un mot, ou un groupe de mots, souvent associés aux noms propres) catégorisables dans des classes telles que noms de personnes, noms d'organisations ou d'entreprises, noms de lieux, quantités, distances, valeurs, dates, etc.
Les typologies et les jeux d'étiquettes sont dépendents de chaque outil.
Dans le cas de Perdido ...

### 4.1 Perdido Geoparser


`Perdido` est une librairie Python pour le geoparsing de texte en français. Le geoparsing se décompose en deux tâches : le geotagging et le geocoding. Le geotagging est similaire à la tâche de reconnaissance des entités nommées avec un focus particulier pour le repérage d'information spatiale. En plus des entités nommées, nous nous intéressons en particuliers aux relations entres ces entités telles que les relations spatiales (distances, topologie, orientation, etc.).
Le geocoding (ou résolution de toponymes) a pour rôle d'attribuer aux entités de lieux des coordonnées géographiques non ambigues.
`Perdido` s'appuie sur une approche hybride principalement construite à base de règles pour la repérage et la classification des entités nommées. La librairie est disponible en 2 versions : une version standard et une version spécialement adaptée pour les articles encyclopédiques.

Dans cette partie nous allons voir comment utiliser `Perdido` pour la reconnaissance d'entités nommées toujours à partir de notre exemple `Arques`.

* Instancier et paramétrer la chaîne de traitement :

In [ ]:
geoparser = Geoparser(version="Encyclopedie")

* Executer la reconnaissance d'entités nommées :

In [ ]:
arques_perdido = geoparser(arques)

Perdido effectuant la tâche de geocoding en plus du NER, le temps de traitement est plus long qu'avec Stanza ou spaCy, du fait de l'interrogation de ressources geographiques externes pour chaque nom de lieu repéré.

* Afficher la liste des entités nommées repérées :

In [ ]:
for ent in arques_perdido.named_entities:
    print(ent.text, ent.tag)

* Afficher de manière graphique les entités nommées avec `displaCy` grâce à la méthode de conversion `to_spacy_doc`:

In [ ]:
displacy.render(arques_perdido.to_spacy_doc(), style="ent", jupyter=True)

* Un rendu similaire mais qui permet de visualiser les entités imbriquées (`style="ent"` -> `style="span"`) :

In [ ]:
displacy.render(arques_perdido.to_spacy_doc(), style="span", jupyter=True)

Cet exemple permet d'illustrer les différences qu'il peut y avoir entre des outils de NER généraliste et ou un outil de geoparsing. On observe ici que Perdido permet une annotation plus fine grâce aux entités imbriquées (ville de, petite rivière) ainsi que le repérage des coordonnées géographiques. En fonction du besoin le repérage de ces éléments peut etre utile pour les traitements suivants ou les analyses qui s'appuient sur ces résultats. 

### 4.2 Expérimentations et comparaison

* Charger l'article `Beaufort` (volume 2, numéro 1365) disponible dans le dossier `data` :

In [ ]:
beaufort = load_txt('data/edda-volume02-1365.txt')

print(beaufort)

* Perdido

In [ ]:
beaufort_perdido = geoparser(beaufort)
displacy.render(beaufort_perdido.to_spacy_doc(), style="ent", jupyter=True)
displacy.render(beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

In [ ]:
lge_beaufort = load_txt('data/lge-beaufort.txt')
print(lge_beaufort)

Cette fois l'article est un peu plus long et comporte des césures de lignes importantes, définissons donc une fonction pour recoller les morceaux :

In [ ]:
def join_lines(s):
    return s.replace('¬\n', '').replace('-\n', '').replace('\n', ' ')

In [ ]:
normalized_lge_beaufort = join_lines(lge_beaufort)
normalized_lge_beaufort

* Perdido

In [ ]:
lge_beaufort_perdido = geoparser(normalized_lge_beaufort)
displacy.render(lge_beaufort_perdido.to_spacy_doc(), style="span", jupyter=True)

## 5. Geoparsing / Geocoding

En complément de la tâche de reconnaissance des entités nommées la librairie `Perdido` propose également celle de résolution des toponymes, on parle alors de *Geoparsing*. Cette tâche consiste a associer à un nom de lieu des coordonnées géographiques non ambigus. De manière classique elle s'appuie sur le repérage des entités spatiales identifées lors de la reconnaissance des entités nommées et fait appel à des ressources externes de type *gazetier* (ou dictionnaires topographique) pour localiser les lieux.

### 5.1 Perdido Geoparser

* Revenons à l'article `ARQUES`

In [ ]:
print(arques)
displacy.render(arques_perdido.to_spacy_doc(), style="ent", jupyter=True)

* En plus de pouvoir afficher la liste des entités nommées comme nous l'avons fait précédemmment, nous pouvons directement afficher la carte des lieux localisés

In [ ]:
# afficher la carte des lieux localisés
arques_perdido.get_folium_map()

Par défaut, lors de l'instanciation du `Geoparser()`, seul [OpenStreetMap](https://www.openstreetmap.org/) est utilisé pour le geocoding et au maximum un résultat est retourné pour chaque lieu (nous verrons dans la suite comment paramétrer le geocoding).

On a déjà ici un aperçu de la difficulté de la tâche de résolution des toponymes. En effet, un grand nombre d'ambiguïtés existent tels que plusieurs lieux ayant le même nom, plusieurs noms pour un même lieu ou encore le fait qu'un lieu ne soit pas référencé dans les ressources que l'on interroge.

### 5.2 Perdido Geocoder

En complément du `Geoparser` qui prend en paramètre un texte et qui fait la reconnaissance d'entités nommées en amont de l'étape de geocoding, `Perdido` propose également une fonction de geocoding disctincte prenant en paramètre directement un nom de lieu (ou une liste de noms de lieux).

In [ ]:
geocoder = Geocoder()
doc = geocoder(['Arques', 'France', 'Normandie', 'Caux'])

# afficher la carte des lieux localisés
doc.get_folium_map()

### 5.3 Résolution de toponymes / désambiguïsation


#### 5.3.1 Exemple : Arques

* Cherchons à localiser la ville `Arques`


In [ ]:
geocoder = Geocoder()
doc = geocoder('Arques')
doc.get_folium_map()

On remarque que par défaut, la localisation retournée pour le nom de lieu `Arques` n'est pas celle que l'on recherche. En effet, le texte indique qu'il s'agit d'une ville de Normandie, or ici la localisation proposée est située dans le Pas-de-Calais !

Changeons les paramètres du `Geocoder` (ces paramètres sont similaires pour le `Geoparser`) pour essayer de retrouver la bonne localisation.

* Augmenter le nombre de résultats retournés par les gazetiers interrogés

In [ ]:
geocoder = Geocoder(max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

On observe parmi les 10 localisations retournées par OpenStreetMap (gazetier par défaut) qu'aucune ne se situe en Normandie.

* Remplacer OpenStreetMap par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'])
doc = geocoder('Arques')
doc.get_folium_map()

On observe que le premier résultat retourné par l'IGN ne se situe ni en Normandie (comme attendu), ni dans le Pas-de-Calais comme le premier résultat retourné par OpenStreetMap.

* Augmenter le nombre de résultats retournés par l'IGN

In [ ]:
geocoder = Geocoder(sources=['ign'], max_rows=10)
doc = geocoder('Arques')
doc.get_folium_map()

Cette fois-ci on retrouve bien une localisation en Normandie au sud de Dieppe avec pour nom `Arques-la-Bataille'. On peut faire l'hypotèse que le nom a évolué car cette localisation se situe bien dans le Pays de Caux (voir illustration ci-dessous, source [Wikipedia](https://fr.wikipedia.org/wiki/Pays_de_Caux)) comme l'indique le texte de l'article.

![Pays de Caux](https://upload.wikimedia.org/wikipedia/commons/thumb/4/4e/Carte_pays_Caux1.png/497px-Carte_pays_Caux1.png)

Ce problème d'ambiguïtés peut aussi être illustrer en allant directement interroger le site web du [géoportail](https://www.geoportail.gouv.fr) comme le montre la capture ci-dessous.

![Résultats sur le géoportail](https://github.com/lmoncla/demo-perdido-egc-2023/blob/main/img/geoportail-arques.png)


Il reste néanmoins le problème de retrouver cette localisation de manière automatique. 
Plusieurs approches existent dans la littérature mais ne sont pas encore implémentées dans `Perdido`.

Cet exemple illustre bien la difficulté de la problématique de désambiguïsation des toponymes avec notamment la gestion des natures de lieux différentes (pays, régions, communes, lieux-dits, lac, rivières, etc.) associés à un même nom, l'homonymie, la non exaustivité des ressources, l'évolution des noms au cours du temps ou encore les erreurs d'orthographe.

* Afficher la carte obtenue après le geoparsing avec l'IGN et 10 résultats max par nom de lieu

In [ ]:
geoparser = Geoparser(sources=['ign'], max_rows=10)
doc = geoparser(arques)
doc.get_folium_map()


### 5.4 Le cas des descriptions de randonnées

Prenons maintenant l'exemple du geoparsing de descriptions de randonnées. Certaines solutions de désambiguisation ont pu être développées et intégrées au sein de la librairie `Perdido` (d'autres sont en cours d'intégration). Les solutions décrites dans la suite de cette partie ont été développées dans le cadre des projets [Perdido](http://erig.univ-pau.fr/PERDIDO/) (2012-2015) et [ANR CHOUCAS](http://choucas.ign.fr) (2017-2022). 

> Ludovic Moncla, Walter Renteria-Agualimpia, Javier Nogueras-Iso and Mauro Gaio (2014). "Geocoding for texts with fine-grain toponyms: an experiment on a geoparsed hiking descriptions corpus". In Proceedings of the 22nd ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, pp 183-192.

> Mauro Gaio and Ludovic Moncla (2019). “Geoparsing and geocoding places in a dynamic space context.“ In The Semantics of Dynamic Space in French: Descriptive, experimental and formal studies on motion expression, 66, 353.


Nous avons choisi un exemple pour illustrer les différentes phases du processus que nous avons mis en place dans le cadre du geoparsing de descriptions de randonnées :
1. filtrer les résultats en fonction du pays 
2. filtrer les résultats en fonction d'une zone géographique définie 
3. regrouper les résultats en utilisant un algorithme de clustering spatial (DBSCAN, *density-based spatial clustering of applications with noise*)
4. selectionner le cluster qui contient le plus d'entités distinctes

La librairie Perdido utilise la méthode DBSCAN implémentée dans la librairie [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html). 
Cette stratégie est adaptée pour une description d'itinéraire où les différents lieux cités sont supposés être localisés à proximité les uns des autres.

* Charger le jeu de données CHOUCAS de descriptions de randonnées fourni par `Perdido`

In [ ]:
dataset_choucas = load_choucas_perdido()
data_choucas = dataset_choucas['data']

data_choucas.to_dataframe().head()

In [ ]:
len(data_choucas)

* Sélectionner une randonnée (parmi les 30)

In [ ]:
id_rando = 2
doc = data_choucas[id_rando]

In [ ]:
doc.text

In [ ]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

In [ ]:
doc.get_folium_map()

On observe ici le résultat déjà pré-traité par `Perdido`. Nous allons maintenant illustrer le processus de désambiguïsation.

On recommence le processus de geoparsing en entier à partir du texte de la randonnées choisie.

In [ ]:
geoparser = Geoparser()
doc_geoparsed = geoparser(doc.text)

In [ ]:
doc_geoparsed.get_folium_map()

On voit clairement la différence par rapport au résultat précédent. Nous allons alors essayer de retrouver le même résultat en déroulant les différentes étapes pour désambiguïser avec `Perdido`.

Pour gagner un peu de temps lors des prochaines executions nous allons faire directement appel à la fonction de geocoding à partir de la liste des noms de lieux.

* Récuperer la liste des noms de lieux (sans doublon)

In [ ]:
places_list = list(set([ent.text for ent in doc_geoparsed.ne_place]))
print(places_list)

#### 5.4.1 Ajout d'un filtre "code pays"


In [ ]:
# instancier le geocoder avec le code pays
geocoder = Geocoder(country_code = 'fr')
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

doc_geocoded.get_folium_map()

#### 5.4.2 Ajout d'un filtre "bounding box"

In [ ]:
bbox = [5.62216508714297, 45.051683489057, 7.18563279407213, 45.9384576816403] # zone d'intervention du PGHM Isère

# instancier le geocoder avec le code pays et une bounding box
geocoder = Geocoder(country_code = 'fr', bbox = bbox)
doc_geocoded = geocoder(places_list)

# ajouter la trace GPS 
doc_geocoded.geometry_layer = doc.geometry_layer

# affiche la carte
doc_geocoded.get_folium_map()

#### 5.4.3 Clustering par densité spatiale

In [ ]:
# appliquer la désambiguïsation 
doc_geocoded.cluster_disambiguation()
doc_geocoded.get_folium_map()